In [ ]:
import os
import numpy as np
import cv2
from glob import glob

height, width = 256, 256  # Image dimensions

def yolo_to_mask(img_shape, bboxes):
    mask = np.zeros(img_shape[:2], dtype=np.uint8)
    for bbox in bboxes:
        class_id, x_center, y_center, w, h = bbox
        x_center, y_center, w, h = x_center * img_shape[1], y_center * img_shape[0], w * img_shape[1], h * img_shape[0]
        x1 = int(x_center - w / 2)
        y1 = int(y_center - h / 2)
        x2 = int(x_center + w / 2)
        y2 = int(y_center + h / 2)
        
        # Debugging output
        print(f"Class ID: {class_id}")
        print(f"Bounding Box (normalized): {bbox}")
        print(f"Bounding Box (pixel): {(x1, y1, x2, y2)}")
        
        # Check if bounding box is within image dimensions
        if x1 < 0 or y1 < 0 or x2 > img_shape[1] or y2 > img_shape[0]:
            print("Bounding box is out of image bounds, skipping this box.")
            continue
        
        mask[y1:y2, x1:x2] = 255  # Set mask to 1 for the region covered by the bounding box
    return mask

def create_masks(data_dir):
    image_dirs = ["train", "val", "test"]
    for dir_name in image_dirs:
        image_dir = os.path.join(data_dir, dir_name)
        mask_dir = os.path.join(data_dir, dir_name, "masks")
        create_dir(mask_dir)
        
        for image_path in glob(os.path.join(image_dir, "*.png")):
            base_name = os.path.basename(image_path).split('.')[0]
            annotation_path = os.path.join(image_dir, base_name + ".txt")
            
            if not os.path.exists(annotation_path):
                continue
            
            img = cv2.imread(image_path)
            img_shape = img.shape

            with open(annotation_path, 'r') as file:
                lines = file.readlines()
                bboxes = []
                for line in lines:
                    class_id, x_center, y_center, w, h = map(float, line.strip().split())
                    bboxes.append((class_id, x_center, y_center, w, h))
            
            mask = yolo_to_mask(img_shape, bboxes)
            mask_path = os.path.join(mask_dir, base_name + "_mask.png")
            cv2.imwrite(mask_path, mask)

def create_dir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)
        print(f"Directory {directory} created.")
    else:
        print(f"Directory {directory} already exists.")

# Path to your data
data_dir = r"F:\Study\TU Dortmund\Industrial Data Science 2\Code\Reference\unet\data"

# Create masks for all images in train, validate, and test folders
create_masks(data_dir)


### Try 2

In [ ]:
import os
import numpy as np
import cv2
from glob import glob

height, width = 256, 256  # Image dimensions

def yolo_to_mask(img_shape, bboxes):
    mask = np.zeros((img_shape[0], img_shape[1], 2), dtype=np.uint8)  # Two channels: one for each class
    for bbox in bboxes:
        class_id, x_center, y_center, w, h = bbox
        x_center, y_center, w, h = x_center * img_shape[1], y_center * img_shape[0], w * img_shape[1], h * img_shape[0]
        x1 = int(x_center - w / 2)
        y1 = int(y_center - h / 2)
        x2 = int(x_center + w / 2)
        y2 = int(y_center + h / 2)
        
        # Debugging output
        print(f"Class ID: {class_id}")
        print(f"Bounding Box (normalized): {bbox}")
        print(f"Bounding Box (pixel): {(x1, y1, x2, y2)}")
        
        # Check if bounding box is within image dimensions
        if x1 < 0 or y1 < 0 or x2 > img_shape[1] or y2 > img_shape[0]:
            print("Bounding box is out of image bounds, skipping this box.")
            continue
        
        # Set mask for the appropriate class
        mask[y1:y2, x1:x2, int(class_id)] = 255
    return mask

def create_masks(data_dir):
    image_dirs = ["train", "val", "test"]
    for dir_name in image_dirs:
        image_dir = os.path.join(data_dir, dir_name)
        mask_dir = os.path.join(data_dir, dir_name, "masks")
        create_dir(mask_dir)
        
        for image_path in glob(os.path.join(image_dir, "*.png")):
            base_name = os.path.basename(image_path).split('.')[0]
            annotation_path = os.path.join(image_dir, base_name + ".txt")
            
            if not os.path.exists(annotation_path):
                continue
            
            img = cv2.imread(image_path)
            img_shape = img.shape

            with open(annotation_path, 'r') as file:
                lines = file.readlines()
                bboxes = []
                for line in lines:
                    class_id, x_center, y_center, w, h = map(float, line.strip().split())
                    bboxes.append((class_id, x_center, y_center, w, h))
            
            mask = yolo_to_mask(img_shape, bboxes)
            
            # Save each class mask separately
            for class_id in range(mask.shape[-1]):
                class_mask_path = os.path.join(mask_dir, f"{base_name}_class_{class_id}_mask.png")
                cv2.imwrite(class_mask_path, mask[:, :, class_id])

def create_dir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)
        print(f"Directory {directory} created.")
    else:
        print(f"Directory {directory} already exists.")

# Path to your data
data_dir = r"/home/user/unet/data"

# Create masks for all images in train, validate, and test folders
create_masks(data_dir)
